# **TCLab Closed-Loop PID with FeedForward**

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors
import time
import tclab

import package_LAB
from importlib import reload
package_LAB = reload(package_LAB)

from package_LAB import LL_RT, PID_RT, IMCTuning
from package_DBR import SelectPath_RT, Delay_RT, FO_RT

In [30]:
#plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import interactive, VBox, IntRangeSlider, IntSlider, Checkbox, interactive_output

## TCLab parameters

In [31]:
#DV 
Kp_ODV_SOPDT = 0.2951290424136788
T1_ODV_SOPDT = 182.2549613489765
T2_ODV_SOPDT = 13.184430234847984
theta_ODV_SOPDT = 28.999891911961512

#MV
Kp_OMV_SOPDT = 0.30788564834253684
T1_OMV_SOPDT = 183.81942938046797
T2_OMV_SOPDT = 3.2920224028341535e-12
theta_OMV_SOPDT = 20.015407110302775

#Operating points 
DV0 = 50 
MV0 = 50
PV0 = 49.3

# Set maximum and minimum MV values
MVmin = 0
MVmax = 100

# Coefficients
alpha = 0.7
gamma = 0.5

#IMC Tuning
Kc, TI, TD = IMCTuning(Kp_OMV_SOPDT, T1_OMV_SOPDT, T2_OMV_SOPDT, theta_OMV_SOPDT, gamma, model="SOPDT")
print(f"Kc: {Kc}, TI: {TI}, TD: {TD}")

# Create a controller
lab = tclab.TCLab()

Kc: 5.33426261068635, TI: 183.81942938047126, TD: 3.2920224028340946e-12
TCLab version 1.0.0
--- Serial Ports ---


RuntimeError: No Arduino device found.

## Arrays, Paths and Variables Initialization

In [ ]:
t = []

SP = []
PV = []
MAN = []
MV_MAN = []
DV = []
MVFF = []
MV = []
MVp = []
MVi = []
MVd = []
E = []
PV_p = []
PV_d = []

MVFF_Delay = []
MVFF_LL1 = []
MV_Delay_P = []
MV_FO_P = []
MV_Delay_D = []
MV_FO_D = []

#default scenario
SPPath = {0: PV0}
ManPath = {0: False}
MVManPath = {0: MV0}
DVPath = {0: DV0}
FF = True
ManFF = False # Not needed

#Time parameters
TSim = 3000
Ts = 1    
N = int(TSim/Ts) + 1 


## PID and plot

In [ ]:
fig = go.FigureWidget(make_subplots(rows=4, cols=1, specs = [[{}], [{}], [{}], [{}]], vertical_spacing = 0.15, row_heights=[0.1, 0.4, 0.4, 0.1], subplot_titles=("Manual Mode", "MV and Components", "PV, SP and E", "Perturbation DV")))
fig.add_trace(go.Scatter(x=t, y=SP, name="SP"), row=3, col=1)
fig.add_trace(go.Scatter(x=t, y=PV, name="PV"), row=3, col=1)
fig.add_trace(go.Scatter(x=t, y=E, name="E", line=dict(dash='dash')), row=3, col=1)
fig.add_trace(go.Scatter(x=t, y=MV, name="MV"), row=2, col=1)
fig.add_trace(go.Scatter(x=t, y=MVp, name="MVp", line=dict(dash='dash')), row=2, col=1)
fig.add_trace(go.Scatter(x=t, y=MVi, name="MVi", line=dict(dash='dash')), row=2, col=1)
fig.add_trace(go.Scatter(x=t, y=MVd, name="MVd", line=dict(dash='dash')), row=2, col=1)
fig.add_trace(go.Scatter(x=t, y=MAN, name="Man"), row=1, col=1)
fig.add_trace(go.Scatter(x=t, y=MV_MAN, name="MVMan"), row=1, col=1)
fig.add_trace(go.Scatter(x=t, y=DV, name="DV"), row=4, col=1)
# Update layout
fig['layout'].update(height=800, width=800)
fig['layout']['xaxis1'].update(title='Time (s)')
fig['layout']['yaxis1'].update(title='(°C)')
fig['layout']['xaxis2'].update(title='Time (s)')
fig['layout']['yaxis2'].update(title='MV (%)')
fig['layout']['xaxis3'].update(title='Time (s)')
fig['layout']['xaxis4'].update(title='Time (s)')


layout.XAxis({
    'anchor': 'y4', 'domain': [0.0, 1.0], 'title': {'text': 'Time (s)'}
})

In [ ]:
def RunExp(Exp):
    if Exp :
        for i in range(0, TSim):
            t.append(i * Ts)
            
            if t[-1] == 0:
                last_time = time.time()
                
            #Process value
            PV.append(lab.T1)
                
            SelectPath_RT(SPPath, t, SP)
            SelectPath_RT(ManPath, t, MAN)
            SelectPath_RT(MVManPath, t, MV_MAN)
            SelectPath_RT(DVPath, t, DV)
            
            #Apply DV
            lab.Q2(DV[-1])
            
            
            # FeedForward
            Delay_RT(DV - DV0*np.ones_like(DV), max(theta_ODV_SOPDT-theta_OMV_SOPDT, 0), Ts, MVFF_Delay)
            LL_RT(MVFF_Delay, -Kp_ODV_SOPDT/Kp_OMV_SOPDT, T1_OMV_SOPDT, T1_ODV_SOPDT, Ts, MVFF_LL1)
            if FF == True:
                LL_RT(MVFF_LL1, 1, T2_OMV_SOPDT, T2_ODV_SOPDT, Ts, MVFF)
            else:
                LL_RT(MVFF_LL1, 0, T2_OMV_SOPDT, T2_ODV_SOPDT, Ts, MVFF) # Set MVFF to 0 if FF is disabled
            
            # PID
            PID_RT(SP, PV, MAN, MV_MAN, MVFF, Kc, TI, TD, alpha, Ts, MVmin, MVmax, MV, MVp, MVi, MVd, E, ManFF, PV0)
            
            #Manipulated value
            lab.Q1(MV[-1])
            
            # wait to the next loop
            elapsed = time.time() - last_time
            time.sleep(max(0, Ts - elapsed))
            last_time = time.time()
            
            fig.data[0].x = t
            fig.data[0].y = SP
            fig.data[1].x = t
            fig.data[1].y = PV
            fig.data[2].x = t
            fig.data[2].y = E
            fig.data[3].x = t
            fig.data[3].y = MV
            fig.data[4].x = t
            fig.data[4].y = MVp
            fig.data[5].x = t
            fig.data[5].y = MVi
            fig.data[6].x = t
            fig.data[6].y = MVd
            fig.data[7].x = t
            fig.data[7].y = MAN
            fig.data[8].x = t
            fig.data[8].y = MV_MAN
            fig.data[9].x = t
            fig.data[9].y = DV
            fig

RunExp(True)


    

NameError: name 'TSim' is not defined